<h1 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
Brief explanation on our Booter database analysis methodology: <br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

# Analysis of: superstresser
#### File originally retrieved from: http://4lz5rmnkd6f63tmm.onion/db/superstresser_db.txt

## * Libraries that I use to analyse data

In [59]:
import pandas as pd
import numpy as np

from io import StringIO
import re

from IPython.display import display,clear_output

<h1 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
STEP 0: Reading an input Booter database file<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

In [8]:
dumpfile='superstresser_db.sql'

<h1 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
STEP 1: Adaptation to our Booter database schema<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

## 1. Reading the first 'N' (100) lines of the input Booter database file

In [9]:
lines_to_read=100

with open(dumpfile) as myfile:
    firstlines=myfile.readlines()[0:lines_to_read] #put here the interval you want
    for x in firstlines:
        print(x.strip())

-- phpMyAdmin SQL Dump
-- version OVH
-- http://www.phpmyadmin.net
--
-- Host: mysql51-113.perso
-- Generation Time: Feb 20, 2014 at 02:12 AM
-- Server version: 5.1.66
-- PHP Version: 5.3.8

SET SQL_MODE="NO_AUTO_VALUE_ON_ZERO";
SET time_zone = "+00:00";

--
-- Database: `superstrzsuper`
--

-- --------------------------------------------------------

--
-- Table structure for table `actions`
--

CREATE TABLE IF NOT EXISTS `actions` (
`ID` int(11) NOT NULL AUTO_INCREMENT,
`username` varchar(255) COLLATE utf8_unicode_ci NOT NULL,
`action` varchar(255) COLLATE utf8_unicode_ci NOT NULL,
`date` varchar(255) COLLATE utf8_unicode_ci NOT NULL,
`userid` varchar(255) COLLATE utf8_unicode_ci NOT NULL,
`url` varchar(255) COLLATE utf8_unicode_ci NOT NULL,
`IP` varchar(255) COLLATE utf8_unicode_ci NOT NULL,
PRIMARY KEY (`ID`)
) ENGINE=MyISAM  DEFAULT CHARSET=utf8 COLLATE=utf8_unicode_ci AUTO_INCREMENT=272 ;

--
-- Dumping data for table `actions`
--

INSERT INTO `actions` (`ID`, `username`, `action

## 2. Listing tables that have content inserted into the dump file

In [10]:
def list_tables_with_insert(dumpfile):
    tables = []
    with open(dumpfile, 'rb') as f:
        for line in f:
            line = line.decode("utf-8").strip()
            if line.lower().startswith('insert'):
                table = re.findall(r'`(.*?)`', line)
                tables.append(table[0])
    return tables

In [24]:
tables=enumerate(list_tables_with_insert(dumpfile))

for i, item in tables:
    print(i+1,item)

1 actions
2 api
3 bans
4 blacklist
5 fe
6 gateway
7 iplogs
8 loginlogs
9 loginlogs
10 loginlogs
11 loginlogs
12 loginlogs
13 loginlogs
14 loginlogs
15 logs
16 logs
17 logs
18 logs
19 logs
20 logs
21 logs
22 logs
23 logs
24 logs
25 logs
26 news
27 payments
28 payments
29 planned
30 plans
31 users
32 users
33 users
34 users
35 users
36 users


## How the list above looks like? 

<h2 align='center'>======================================================================
If NOT well-formed SQL dump file then you must first do the following:
======================================================================</h2>

## 3. Removing lines that are not part of the actual content to be analysed

## 4. Splitting and naming tables and columns

<h2 align='center'>======================================================================
Additional functions
======================================================================</h2>

## * Function to read tables from wel-formed SQL database dump

In [12]:
# For well formed SQL database dumps!
def read_inserted_table(dumpfile, target_table):
    sio = StringIO()
    fast_forward = True
    already_header = False
    with open(dumpfile, 'rb') as f:
        for line in f:
            line = line.decode("utf-8").strip()
            line = re.sub("(?!(([^']*'){2})*[^']*$)\)", '',line) #Step0:remove any ")" from the content of columns
            if line.lower().startswith('insert') and target_table in line:
                fast_forward = False
                if already_header:
                    continue
            if fast_forward:
                continue
            
            data = re.findall('\([^\)]*\)', line) #Step1: get the content between parentesis (i.e., insert line)
            try:
                newline = data[0].strip('()') #Step2:remove parenthesis
                newline=newline.replace('`','') #Step3: remove ` (usually in table names)
                newline=re.sub(r'(?!(([^\']*\'){2})*[^\']*$),','', newline) #Step4: remove commas from the content of columns
                newline=newline.replace('\'','') #Step5: remove single quotes
                newline=newline.replace(', ', ',') #Step6: remove single spaces after comma (i.e., in the beginning of a column)
                sio.write(newline)
                sio.write("\n")
            except IndexError:
                pass
            if line.endswith(';'):
                fast_forward = True
                already_header = True
    sio.seek(0)
    return sio

## * Converter functions for formatting content of columns
Attention, it is better to perform the conversion based on a series than a string (http://stackoverflow.com/questions/42462906/pandas-read-csv-converters-performance-issue)

In [13]:
def tolowercase(series):
    return series.str.lower()

In [14]:
def strip_singlequote(series):
    return series.str.strip('\'')

In [15]:
def timestamp2datetime(series):
    return  pd.to_datetime(series,unit='s',errors='coerce')

In [16]:
def datetimestring2datetime(series):
    return  pd.to_datetime(series, format='%d-%m-%Y %H:%M',errors='coerce')  

In [17]:
def datetimeinvertedstring2datetime(series):
    return pd.to_datetime(series, format='%Y-%m-%d %H:%M:%S',errors='coerce')

In [18]:
def datetimeampm2datetime(series):
    return  pd.to_datetime(series,format='%d-%m-%Y %I:%M:%S %p',errors='coerce')

In [19]:
def datetimemdyampm2datetime(series):
    return  pd.to_datetime(series, format='%m-%d-%Y %I:%M:%S %p',errors='coerce') 

In [20]:
def strip_parenthesisandsemicolon(series):
    return series.str.strip(');')

In [21]:
def strip_insertintologs(series):
    return int(series.str.strip('INSERT INTO `logs` VALUES ('))

In [22]:
def strip_tab(series):
    return series.str.strip('\t')

In [53]:
def friendsenemies_type(series):
    return series.str.replace('f','friend').replace('e','enemy')
    

<h2 align='center'>======================================================================
Adapting EACH existing table
======================================================================</h2>

## 6.1. Table: 'actions'

### How this table looks like without modification?

In [25]:
tablename='actions'

pd.read_csv(read_inserted_table(dumpfile,tablename),delimiter=",",error_bad_lines=False)

,ID,username,action,date,userid,url,IP
0,124,Xepher,The Username Of sustephen Has Been Changed To ...,1384204955,4005,http://superstresser.com/admin/edituser.php?id...,108.180.2.107
1,125,Xepher,Membership Of sustephen Has Been Edited From :...,1384204955,4005,http://superstresser.com/admin/edituser.php?id...,108.180.2.107
2,126,Xepher,The Username Of Dragonborn Has Been Changed To...,1384221132,4005,http://superstresser.com/admin/edituser.php?id...,108.180.2.107
3,127,Xepher,Dragonborn Rank Has Been Edited To 0,1384221132,4005,http://superstresser.com/admin/edituser.php?id...,108.180.2.107
4,128,Matt,The Username Of Xepher Has Been Changed To Xepher,1384221246,587,http://www.superstresser.com/admin/edituser.ph...,72.48.203.140
5,129,Matt,Xepher Rank Has Been Edited To 1,1384221246,587,http://www.superstresser.com/admin/edituser.ph...,72.48.203.140
6,130,Matt,The Username Of Xepher Has Been Changed To Xepher,1384221287,587,http://www.superstresser.com/admin/edituser.ph...,72.48.203.140
7,131,Matt,Xepher Rank Has Been Edited To 1,1384221287,587,http://www.superstresser.com/admin/edituser.ph...,72.48.203.140
8,132,Matt,The Username Of Xepher Has Been Changed To Xepher,1384221301,587,http://www.superstresser.com/admin/edituser.ph...,72.48.203.140
9,133,Matt,Xepher Rank Has Been Edited To 1,1384221301,587,http://www.superstresser.com/admin/edituser.ph...,72.48.203.140


#### Q1: Is this table different from other previous loaded table? YES
#### Q2: Is this table similar to any table in the generic Booter database schema? NO
    - This table looks like: 
    
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type/converter: 
        o 
    - On the column name:
        o 
    - Split columns:
        o 
    - Add required columns:
        o 

## 6.2. Table: 'api'

### How this table looks like without modification?

In [26]:
tablename='api'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,ID,URL,Note
0,1,http://94.102.51.117/api/send.php?key=OUG7FA8G...,Alpha
1,3,http://89.248.174.2/api/send.php?key=OUG7FA8GD...,Charlie


#### Q1: Is this table different from other previous loaded table? YES
#### Q2: Is this table similar to any table in the generic Booter database schema? YES 
    - This table looks like: webshells
    
#### Q3: Are there modifications required? 
    - On the table name: 
        o api -> webshells
    - On the column type/converter: 
        o 
    - On the column name:
        o ID -> id
        o URL -> url
        o 
    - Split columns:
        o 
    - Add required columns:
        o lastchecked
        o status
        o attacktype

### Modifying table: 'boot-log'

In [27]:
# Loading the column converting columns with predefined functions
df_webshells = pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False)

# Converters                                     


# Changing names of columns
df_webshells.rename(inplace=True,columns = {'ID':'id',
                                            'URL': 'url'})

# Creating empty columns (with "" [for future string] or np.nan [for future float])
df_webshells['lastchecked'] = ''
df_webshells['status'] = ''
df_webshells['attacktype'] = ''

# Showing some lines after adapt the table
df_webshells.head(10)

,id,url,Note,lastchecked,status,attacktype
0,1,http://94.102.51.117/api/send.php?key=OUG7FA8G...,Alpha,,,
1,3,http://89.248.174.2/api/send.php?key=OUG7FA8GD...,Charlie,,,


## 6.3. Table: 'bans'

### How this table looks like without modification?

In [28]:
tablename='bans'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,id,username,banreason,bannedby,timestamp
0,2,chuddyni,Attacking same IP,killermanjaro65,1378670549
1,3,legionerebos,Attacking same IP,killermanjaro65,1378781464
2,4,nathanjo,Attacking same IP,killermanjaro65,1379030455
3,6,uziuser,PayPal chargeback,Egregious,1380735628
4,7,rockandrave,PayPal chargeback,Egregious,1380735716
5,8,Paixo,Unauthorized user,Egregious,1381117501
6,9,dark1g,PayPal chargeback,Egregious,1381143658
7,10,Scondy,Multiple PayPal chargebacks,Egregious,1381352225
8,11,InstantClear,Nice lies faggot go suck a gaping black cock a...,Dragonborn,1381633275
9,12,cipher,Spam botting the user list and chat support.,Dragonborn,1381961904


#### Q1: Is this table different from other previous loaded table? YES
#### Q2: Is this table similar to any table in the generic Booter database schema? NO
    - This table looks like: 
    
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o    
    - On the column name:
        o  
    - Split columns:
        o 
    - Add required columns:
        o     

## 6.4. Table: 'blacklist'

### How this table looks like without modification?

In [29]:
tablename='blacklist'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,ID,IP,note
0,1,127.0.0.1,Localhost
1,3,184.75.107.144,Lawrence Wong
2,12,173.194.41.66,Youtube
3,17,64.222.131.9,Jeff Ward
4,6,198.81.129.107,CIA
5,7,173.223.104.110,White House
6,8,173.252.110.27,Facebook
7,9,72.21.81.85,FBI
8,10,141.101.123.216,Cloudflare
9,11,173.194.64.102,Google


#### Q1: Is this table different from other previous loaded table? YES
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: blacklist
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o 
    - On the column name:
        o ID -> id
        o IP -> ip
    - Add required columns:
        o  
    - Split columns:
        o 

### Modifying table: 'blacklist'

In [30]:
# Loading the correct table name
df_blacklist = pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False)

# Converting columns

# Changing column names
df_blacklist.rename(inplace=True, columns = {'ID': 'id',
                                            'IP':'ip'})

# Adding missing columns (with "" [for future string] or np.nan [for future float])

###Showing some lines after adapt the table
df_blacklist.head(10)

,id,ip,note
0,1,127.0.0.1,Localhost
1,3,184.75.107.144,Lawrence Wong
2,12,173.194.41.66,Youtube
3,17,64.222.131.9,Jeff Ward
4,6,198.81.129.107,CIA
5,7,173.223.104.110,White House
6,8,173.252.110.27,Facebook
7,9,72.21.81.85,FBI
8,10,141.101.123.216,Cloudflare
9,11,173.194.64.102,Google


## 6.5. Table: 'fe'

### How this table looks like without modification?

In [31]:
tablename='fe'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,ID,userid,type,ipaddr,note
0,3,5944,e,192.185.228.222,NaN
1,4,6535,e,78.57.241.253,Naranoz
2,5,6535,e,78.61.177.211,Dariušas
3,6,6539,e,107.211.178.121,Nicko4572 hate him he thinks i hack ~xB
4,8,6535,e,78.56.188.93,Du Duchai
5,9,6474,e,72.20.8.172,NaN
6,10,6535,e,5.20.76.87,2
7,11,6227,f,24.43.157.18,SMPS PUBLIC IP\r\n
8,12,6227,e,65.52.24.110,Xeccution IP DDOS
9,13,6227,f,70.27.107.64,TheGam3Report


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: friendsenemies
#### Q3: Are there modifications required? 
    - On the table name: 
        o fe -> friendsenemies
    - On the column type: 
        o type -> friendsenemies_type() 
    - On the column name:
        o ID -> id
        o ipaddr -> ip
    - Add required columns:
        o 
    - Split columns:
        o 

### Modifying table: 'fe'

In [55]:
# Loading the correct table name
df_friendenemies = pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False)

# Converting columns
df_friendenemies['type']=friendsenemies_type(df_friendenemies['type'])

# Changing column names
df_friendenemies.rename(inplace=True, columns = {'ID': 'id',
                                            'ipaddr':'ip'})

# Adding missing columns (with "" [for future string] or np.nan [for future float])

###Showing some lines after adapt the table
df_friendenemies.head(10)

,id,userid,type,ip,note
0,3,5944,enemy,192.185.228.222,NaN
1,4,6535,enemy,78.57.241.253,Naranoz
2,5,6535,enemy,78.61.177.211,Dariušas
3,6,6539,enemy,107.211.178.121,Nicko4572 hate him he thinks i hack ~xB
4,8,6535,enemy,78.56.188.93,Du Duchai
5,9,6474,enemy,72.20.8.172,NaN
6,10,6535,enemy,5.20.76.87,2
7,11,6227,friend,24.43.157.18,SMPS PUBLIC IP\r\n
8,12,6227,enemy,65.52.24.110,Xeccution IP DDOS
9,13,6227,friend,70.27.107.64,TheGam3Report


## 6.6. Table: 'gateway'

### How this table looks like without modification?

In [56]:
tablename='gateway'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,email
0,lawrencewong9000@gmail.com


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: gateways
#### Q3: Are there modifications required? 
    - On the table name: 
        o gateway -> gateways
    - On the column type: 
        o  
    - On the column name:
        o 
    - Add required columns:
        o 
    - Split columns:
        o 

### Modifying table: 'gateway'

In [57]:
# Loading the correct table name
df_gateways = pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False)

# Converting columns

# Changing column names

# Showing some lines after adapt the tabledf_plans['maxboottime'] = np.nan

# Showing some lines after adapt the table
df_gateways.head(10)

,email
0,lawrencewong9000@gmail.com


## 6.7. Table: 'iplogs'

### How this table looks like without modification?

In [58]:
tablename='iplogs'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,ID,userID,logged,date
0,6,4,151.224.90.88,1364836851
1,29,16,173.252.73.116,1365310723
2,30,16,70.173.45.137,1365310747
3,31,16,70.189.238.210,1365310799
4,32,16,70.189.238.210,1365310957
5,33,16,70.189.238.210,1365311019
6,40,29,86.33.65.88,1366718939
7,35,12,86.144.105.145,1365361982
8,36,21,24.11.92.38,1365481722
9,207,220,166.137.191.36,1374880143


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: logins
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o date -> timestamp2datetime()
    - On the column name:
        o ID -> id
        o userID -> userid
        o logged -> userip
    - Add required columns:
        o username
    - Split columns:
        o 

### Modifying table: 'iplogs'

In [60]:
# Loading the correct table name
df_logins = pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False)

# Converting columns
df_logins['date'] = timestamp2datetime(df_logins['date']) 

# Changing names of columns
df_logins.rename(inplace=True, columns = {'ID': 'id',
                                         'userID': 'userid',
                                         'logged':'userip'})

# Creating empty columns (with "" [for future string] or np.nan [for future float])
df_logins['username']=''


# Showing some lines after adapt the table
df_logins.head(10)

,id,userid,userip,date,username
0,6,4,151.224.90.88,2013-04-01 17:20:51,
1,29,16,173.252.73.116,2013-04-07 04:58:43,
2,30,16,70.173.45.137,2013-04-07 04:59:07,
3,31,16,70.189.238.210,2013-04-07 04:59:59,
4,32,16,70.189.238.210,2013-04-07 05:02:37,
5,33,16,70.189.238.210,2013-04-07 05:03:39,
6,40,29,86.33.65.88,2013-04-23 12:08:59,
7,35,12,86.144.105.145,2013-04-07 19:13:02,
8,36,21,24.11.92.38,2013-04-09 04:28:42,
9,207,220,166.137.191.36,2013-07-26 23:09:03,


## 6.8. Table: 'loginlogs'

### How this table looks like without modification?

In [71]:
tablename='loginlogs'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,id,ipaddr,username,date
0,1,71.161.122.54,killermanjaro65,1390958460
1,2,65.30.166.15,Trippy,1390958732
2,3,71.161.122.54,killermanjaro65,1390958866
3,4,71.183.54.96,Ecutch13,1390959006
4,5,46.165.208.203,null3d,1390959105
5,6,2.83.54.50,xdtugapt,1390959186
6,7,2.83.54.50,xdtugapt,1390959195
7,8,74.106.119.196,getsmashed,1390959245
8,9,74.106.119.196,KaliKush,1390959275
9,10,47.17.154.70,Flixotic,1390959496


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? YES append logins
    - This table looks like: logins
#### Q3: Are there modifications required? 
    - On the table name: 
        o loginlogs -> logins_temp
    - On the column type: 
        o date -> timestamp2datetime()
    - On the column name:
        o ipaddr -> userip
    - Add required columns:
        o userid
    - Split columns:
        o 

### Modifying table: 'loginlogs'

In [72]:
# Loading the correct table name
df_logins_temp = pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False)

# Converting columns
df_logins_temp['date'] = timestamp2datetime(df_logins_temp['date']) 

# Changing names of columns
df_logins_temp.rename(inplace=True, columns = {'ipaddr':'userip'})

# Creating empty columns (with "" [for future string] or np.nan [for future float])
df_logins_temp['userid']=''


# Showing some lines after adapt the table
df_logins_temp.head(10)

,id,userip,username,date,userid
0,1,71.161.122.54,killermanjaro65,2014-01-29 01:21:00,
1,2,65.30.166.15,Trippy,2014-01-29 01:25:32,
2,3,71.161.122.54,killermanjaro65,2014-01-29 01:27:46,
3,4,71.183.54.96,Ecutch13,2014-01-29 01:30:06,
4,5,46.165.208.203,null3d,2014-01-29 01:31:45,
5,6,2.83.54.50,xdtugapt,2014-01-29 01:33:06,
6,7,2.83.54.50,xdtugapt,2014-01-29 01:33:15,
7,8,74.106.119.196,getsmashed,2014-01-29 01:34:05,
8,9,74.106.119.196,KaliKush,2014-01-29 01:34:35,
9,10,47.17.154.70,Flixotic,2014-01-29 01:38:16,


In [73]:
df_logins=df_logins.append(df_logins_temp)

## 6.9. Table: 'logs'!!!!!! PROBLEM mix between login and attack logs

### How this table looks like without modification?

In [75]:
tablename='logs'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False,header=None,names=range(9))

,0,1,2,3,4,5,6,7,8
0,ID,userID,logged,date,NaN,NaN,NaN,NaN,NaN
1,6,4,151.224.90.88,1364836851,NaN,NaN,NaN,NaN,NaN
2,29,16,173.252.73.116,1365310723,NaN,NaN,NaN,NaN,NaN
3,30,16,70.173.45.137,1365310747,NaN,NaN,NaN,NaN,NaN
4,31,16,70.189.238.210,1365310799,NaN,NaN,NaN,NaN,NaN
5,32,16,70.189.238.210,1365310957,NaN,NaN,NaN,NaN,NaN
6,33,16,70.189.238.210,1365311019,NaN,NaN,NaN,NaN,NaN
7,40,29,86.33.65.88,1366718939,NaN,NaN,NaN,NaN,NaN
8,35,12,86.144.105.145,1365361982,NaN,NaN,NaN,NaN,NaN
9,36,21,24.11.92.38,1365481722,NaN,NaN,NaN,NaN,NaN


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? NO
    - This table looks like: users
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o 
    - On the column name:
        o email -> useremail 
        o membership -> planid
    - Add required columns:
        o userid
        o expire
    - Split columns:
        o df_logins['username']=df_users['username']
        o df_logins['userip']=df_users['ip']
        o df_logins['date']=timestamp2datetime(df_users['lastlogin'])

## 6.10. Table: 'news'

### How this table looks like without modification?

In [76]:
tablename='news'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,ID,title,detail,date
0,148,Power Upgrade,The faulty server that we took down a few days...,1392701822
1,142,Xbox One,Here is a special offer for Super Stresser mem...,1391840945


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? NO
    - This table looks like: 
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o 
    - On the column name:
        o 
    - Add required columns:
        o 
    - Split columns:
        o 

## 6.11. Table: 'payments'

### How this table looks like without modification?

In [77]:
tablename='payments'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,ID,paid,plan,user,email,tid,date
0,4,1,1,46,chaseh96@gmail.com,36X9850631614812W,1369436953
1,5,4,2,47,legion_erebos@Live.com,9P507110SY176180R,1369437288
2,6,8,13,52,mitchellprice1@hotmail.com,3DN29639VM1468741,1369453568
3,7,8,13,53,kimdrey@yahoo.com,4FU49980S78412924,1369454289
4,8,1,1,54,nixhostsnet@gmail.com,0GA00576X5704635H,1369455938
5,9,4,2,56,hoopi2mail@gmail.com,7SH3990814297631A,1369483811
6,10,1,1,57,guyon_cornelissen@hotmail.com,38V57873CR977461E,1369494176
7,11,1,1,58,kingrovlopad@gmail.com,7L403762TY8688800,1369500693
8,12,1,1,59,itochkids@yahoo.co.uk,0U128165J9131012H,1369513788
9,13,1,1,60,infinitygrief@gmail.com,55D71084U6992021D,1369526012


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? YES 
    - This table looks like: payments
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o date -> timestamp2datetime()
    - On the column name:
        o ID -> id
        o user -> userid
        o paid -> amountpaid 
        o email -> paymentemail 
        o plan -> planid
    - Add required columns:
        o username
    - Split columns:
        o 

### Modifying table: 'payments'

In [78]:
# Loading the correct table name
df_payments = pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False)

# Converting columns
df_payments['date'] = timestamp2datetime(df_payments['date']) 

# Changing names of columns
df_payments.rename(inplace=True, columns = {'ID':'id',
                                           'user':'userid',
                                           'paid': 'amountpaid',
                                           'email': 'paymentemail',
                                           'plan':'planid'})

# Creating empty columns (with "" [for future string] or np.nan [for future float])
df_payments['username']=''


# Showing some lines after adapt the table
df_payments.head(10)

,id,amountpaid,planid,userid,paymentemail,tid,date,username
0,4,1,1,46,chaseh96@gmail.com,36X9850631614812W,2013-05-24 23:09:13,
1,5,4,2,47,legion_erebos@Live.com,9P507110SY176180R,2013-05-24 23:14:48,
2,6,8,13,52,mitchellprice1@hotmail.com,3DN29639VM1468741,2013-05-25 03:46:08,
3,7,8,13,53,kimdrey@yahoo.com,4FU49980S78412924,2013-05-25 03:58:09,
4,8,1,1,54,nixhostsnet@gmail.com,0GA00576X5704635H,2013-05-25 04:25:38,
5,9,4,2,56,hoopi2mail@gmail.com,7SH3990814297631A,2013-05-25 12:10:11,
6,10,1,1,57,guyon_cornelissen@hotmail.com,38V57873CR977461E,2013-05-25 15:02:56,
7,11,1,1,58,kingrovlopad@gmail.com,7L403762TY8688800,2013-05-25 16:51:33,
8,12,1,1,59,itochkids@yahoo.co.uk,0U128165J9131012H,2013-05-25 20:29:48,
9,13,1,1,60,infinitygrief@gmail.com,55D71084U6992021D,2013-05-25 23:53:32,


## 6.12. Table: 'planned'

### How this table looks like without modification?

In [79]:
tablename='planned'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,id,complete,username,ip,name,boot,time,port,method
0,1,1,Egregious,1.1.1.1,Testing,12,1390967836,80,AUDP
1,2,1,Egregious,1.1.1.1,ONe,12,1390968441,80,AUDP


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? NO
    - This table looks like: 
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o 
    - On the column name:
        o 
    - Add required columns:
        o 
    - Split columns:
        o 

## 6.13. Table: 'plans'

### How this table looks like without modification?

In [80]:
tablename='plans'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,ID,name,description,mbt,unit,length,price
0,1,Trial Membership,Daily - 300 seconds,300,Day,1,1
1,2,Tier 1 Membership,Monthly - 600 seconds,600,Month,1,4
2,3,Tier 2 Membership,Monthly - 1800 seconds,1800,Month,1,6
3,13,Tier 3 Membership,Monthly - 3600 seconds,3600,Months,1,8
4,17,Tier 4 VIP Membership,Monthly - 5400 seconds,5400,Months,1,15
5,18,Tier 5 VIP Membership,Monthly - 7200 seconds,7200,Months,1,20


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? YES 
    - This table looks like: plans
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o 
    - On the column name:
        o ID -> planid
        o name -> planname 
        o description -> plandescr
        o mbt -> maxboottime 
    - Add required columns:
        o concurrecy
    - Split columns:
        o 

### Modifying table: 'plans'

In [81]:
# Loading the correct table name
df_plans = pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False)

# Converting columns
df_plans['date'] = timestamp2datetime(df_payments['date']) 

# Changing names of columns
df_plans.rename(inplace=True, columns = {'ID':'planid',
                                        'name':'planname',
                                        'description':'plandescr',
                                        'mbt': 'maxboottime'})

# Creating empty columns (with "" [for future string] or np.nan [for future float])
df_plans['concurrency']=''


# Showing some lines after adapt the table
df_plans.head(10)

,planid,planname,plandescr,maxboottime,unit,length,price,date,concurrency
0,1,Trial Membership,Daily - 300 seconds,300,Day,1,1,2013-05-24 23:09:13,
1,2,Tier 1 Membership,Monthly - 600 seconds,600,Month,1,4,2013-05-24 23:14:48,
2,3,Tier 2 Membership,Monthly - 1800 seconds,1800,Month,1,6,2013-05-25 03:46:08,
3,13,Tier 3 Membership,Monthly - 3600 seconds,3600,Months,1,8,2013-05-25 03:58:09,
4,17,Tier 4 VIP Membership,Monthly - 5400 seconds,5400,Months,1,15,2013-05-25 04:25:38,
5,18,Tier 5 VIP Membership,Monthly - 7200 seconds,7200,Months,1,20,2013-05-25 12:10:11,


## 6.14. Table: 'users'

### How this table looks like without modification?

In [84]:
tablename='users'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,ID,username,password,email,rank,membership,expire,status,vip,cons
0,336,Rzeio,84c16c013459bd85fc39d6c0e98cc27f93fe3aec,ImJdsGbMnj@yahoo.com,0,0,1377667150,0,0,1
1,6515,snils,7bf29a335b2d027b09580b99d9cb58469c42a1d3,ilovesammyferret@hotmail.com,0,0,0,0,0,1
2,60,parment,5d7211905cd8b6e81ea7cade77a5d9a9bfdcef09,taldond@gmail.com,0,1,1370908412,0,0,1
3,39,rxqueen,41713d315a1f2d99a809bd3ec2f32c3b4eb36c2c,ok@ok.com,0,0,0,0,0,1
4,41,xJHA,0fc4bf953ecbbff0124a9d1c5531ca5d10094231,javersano98@gmail.com,0,0,1372109088,0,0,1
5,49,zach,4967e07bc327c714541206122152ddfcfdb5b27c,divoured@fbi.gov,0,0,0,0,0,1
6,44,RICHARDANG,7fc1398a8d1f06b1fbd771f39b1a9241e21a228b,rda1979wax@yahoo.com,0,0,1381585850,0,0,1
7,45,ikosayi,2f5c98e366de1657b14706cf6f57410a2b07b297,FFlazhHF@gmail.com,0,0,0,0,0,1
8,46,Choos,e87e1f969416ad600bba93d991e0e57d21a7dc23,chaeshult@gmail.com,0,1,1370819353,0,0,1
9,47,legionerebos,a15f0aaad1c409e7b888c1c3bd2eaaa0880281fb,legion_erebos@live.com,0,1,1378856052,1,0,1


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? YES 
    - This table looks like: users
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o expire -> timestamp2datetime()
    - On the column name:
        o ID -> userid
        o email -> useremail 
        o membership -> planid
    - Add required columns:
        o 
    - Split columns:
        o 

### Modifying table: 'users'

In [85]:
# Loading the correct table name
df_users = pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False)

# Converting columns
df_users['expire'] = timestamp2datetime(df_users['expire']) 

# Changing names of columns
df_users.rename(inplace=True, columns = {'ID':'userid',
                                           'email':'useremail',
                                           'membership': 'planid'})

# Creating empty columns (with "" [for future string] or np.nan [for future float])

# Showing some lines after adapt the table
df_users.head(10)

,userid,username,password,useremail,rank,planid,expire,status,vip,cons
0,336,Rzeio,84c16c013459bd85fc39d6c0e98cc27f93fe3aec,ImJdsGbMnj@yahoo.com,0,0,2013-08-28 05:19:10,0,0,1
1,6515,snils,7bf29a335b2d027b09580b99d9cb58469c42a1d3,ilovesammyferret@hotmail.com,0,0,1970-01-01 00:00:00,0,0,1
2,60,parment,5d7211905cd8b6e81ea7cade77a5d9a9bfdcef09,taldond@gmail.com,0,1,2013-06-10 23:53:32,0,0,1
3,39,rxqueen,41713d315a1f2d99a809bd3ec2f32c3b4eb36c2c,ok@ok.com,0,0,1970-01-01 00:00:00,0,0,1
4,41,xJHA,0fc4bf953ecbbff0124a9d1c5531ca5d10094231,javersano98@gmail.com,0,0,2013-06-24 21:24:48,0,0,1
5,49,zach,4967e07bc327c714541206122152ddfcfdb5b27c,divoured@fbi.gov,0,0,1970-01-01 00:00:00,0,0,1
6,44,RICHARDANG,7fc1398a8d1f06b1fbd771f39b1a9241e21a228b,rda1979wax@yahoo.com,0,0,2013-10-12 13:50:50,0,0,1
7,45,ikosayi,2f5c98e366de1657b14706cf6f57410a2b07b297,FFlazhHF@gmail.com,0,0,1970-01-01 00:00:00,0,0,1
8,46,Choos,e87e1f969416ad600bba93d991e0e57d21a7dc23,chaeshult@gmail.com,0,1,2013-06-09 23:09:13,0,0,1
9,47,legionerebos,a15f0aaad1c409e7b888c1c3bd2eaaa0880281fb,legion_erebos@live.com,0,1,2013-09-10 23:34:12,1,0,1


<h2 align='center'>======================================================================
<br>Final step of the manual part
======================================================================</h2>

## 7. Adding missing tables accordingly to our generic Booter database 

In [86]:
df_attacks= pd.DataFrame(columns=['id','userid','username','targetip','targeturl','duration','port','type','date'])
#df_blacklist=pd.DataFrame(columns=['id','ip','note'])
#df_friendsenemies=pd.DataFrame(columns=['id','ip','note','userid','type'])
#df_gateways=pd.DataFrame(columns=['email'])
#df_logins=pd.DataFrame(columns=['id','userid','username','userip','date'])
#df_payments=pd.DataFrame(columns=['id','userid','username','amountpaid','paymentemail','planid','tid','date'])
#df_plans=pd.DataFrame(columns=['planid','planname','plandescr','price','maxboottime','concurrency'])
df_servers=pd.DataFrame(columns=['id','ip'])
df_settings=pd.DataFrame(columns=['url','sitename','siteemail'])
#df_users=pd.DataFrame(columns=['userid','username','useremail','password','expire','plan'])      
#df_webshells=pd.DataFrame(columns=['id','url','status','lastchecked','attacktype'])

<h1 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
STEP 2: Data Enrichment<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

### * Depending of the size of the data, this part can take HOURS. I tested for both small and big datasets and it worked. Be pacient. This will pay-off!

## * Libraries needed to retrieve information from external databases

In [ ]:
import subprocess
import cfscrape
from lxml import etree
import os.path
import random
import time 

## * Function to enrich IP addresseswith AS information and country 

In [ ]:
# THANKS TO: team-cymru.org
def iptoasn(iplist_teamcymruformat_filelocation,outputfile):
    cat = subprocess.Popen(['cat', iplist_teamcymruformat_filelocation], 
                            stdout=subprocess.PIPE)
    
    netcat = subprocess.Popen(['netcat', 'whois.cymru.com', '43'],
                              stdin=cat.stdout,
                              stdout=outputfile)
    time.sleep(3) #for some reason the poll does not work! This was the way to overcome the waiting time.
    
    return netcat.stdout      

## * Function to check if an IP address was Tor node in a given moment

In [ ]:
# THANKS TO: exonerator.torproject.org
def WasTorNode(ip, date ):
    url="https://exonerator.torproject.org/?ip="+ip+"&timestamp="+date
    scraper = cfscrape.create_scraper()
    scraped_html=scraper.get(url).content    
    html_tree = etree.HTML(scraped_html)
    result=html_tree.xpath("//h3[@class='panel-title']/text()") # I was looking for <h3 class="panel-title">Result is positive</h3>
    tor_node=True if result == ['Result is positive'] else False
    return tor_node 
# 'date' MUST BE formated as: Year-month-day (2016-03-31)

## * Discovering the middle date of the database

In [ ]:
try:
    try:
        middle_date=(min(df_attacks['date'])+((max(df_attacks['date'])-min(df_attacks['date']))/2))
        raise
    except Exception:
        pass

    try:
        middle_date=(min(df_logins['date'])+((max(df_logins['date'])-min(df_logins['date']))/2))
        raise
    except Exception:
        pass

    try:
        middle_date=(min(df_payments['date'])+((max(df_payments['date'])-min(df_payments['date']))/2))
        raise
    except:
        pass
except Exception:
    print("There is no date in the entire dataset")

date_tor_check = middle_date.strftime('%Y-%m-%d')
date_iptoasn_lookup= str(middle_date)
print(date_tor_check)

## * Preparing to Perform IP to ASN info

In [ ]:
df_logins['middledate']=date_iptoasn_lookup
df_attacks['middledate']=date_iptoasn_lookup
df_friendsenemies['middledate']=date_iptoasn_lookup
df_blacklist['middledate']=date_iptoasn_lookup

## 2.1.1  Lookup IP to ASN info of table: logins

In [ ]:
if (os.path.exists('enrichments/logins_iptoasn_out')== False):
    logins_iptoasn_in = open('enrichments/logins_iptoasn_in', 'w+')
    logins_iptoasn_in.write('begin\nverbose\n')
    df_logins[['userip','middledate']].drop_duplicates().to_csv(logins_iptoasn_in,header=False,index=False,sep="\t") 
    logins_iptoasn_in.write('end')
    logins_iptoasn_in.close()

    logins_iptoasn_out = open('logins_iptoasn_out', 'w+')
    iptoasn('logins_iptoasn_in',logins_iptoasn_out)
    logins_iptoasn_out.close()
else:
    print("You already performed the lookup for this table")

In [ ]:
df_logins_iptoasn = pd.read_csv('enrichments/logins_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])

df_logins_extended= pd.merge(df_logins,
                              df_logins_iptoasn,
                              how = 'left',
                              left_on = 'userip',
                              right_on = 'ip')

# Changing name of columns to avoid misunderstandings
df_logins_extended.rename(columns={'asn':'srcasn', 
                                   'ip':'srcip', 
                                   'bgp_prefix':'srcbgp_prefix', 
                                   'country':'srccountry' ,
                                   'registry':'srcregistry',
                                   'info_date':'srcinfo_date',
                                   'info_request':'srcinfo_request'},
                         inplace=True)


df_logins_extended.head(5)

## 2.1.2  Lookup IP to ASN info of table: attacks

In [ ]:
if (os.path.exists('enrichments/attacks_iptoasn_out')== False):
    attacks_iptoasn_in = open('enrichments/attacks_iptoasn_in', 'w+')
    attacks_iptoasn_in.write('begin\nverbose\n')
    df_attacks[['targetip','middledate']].drop_duplicates().to_csv(attacks_iptoasn_in,header=False,index=False,sep="\t") 
    attacks_iptoasn_in.write('end')
    attacks_iptoasn_in.close()

    attacks_iptoasn_out = open('attacks_iptoasn_out', 'w+')
    iptoasn('attacks_iptoasn_in',attacks_iptoasn_out)
    attacks_iptoasn_out.close()
else:
    print("You already performed the lookup for this table")

In [ ]:
df_attacks_iptoasn = pd.read_csv('enrichments/attacks_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])

# Merging the iptoasn with the queried column
df_attacks_extended= pd.merge(df_attacks,
                              df_attacks_iptoasn,
                              how = 'left',
                              left_on = 'targetip',
                              right_on = 'ip')

# Changing name of columns to avoid misunderstandings
df_attacks_extended.rename(columns={'asn':'targetasn', 
                                   'ip_y':'targetip', 
                                   'bgp_prefix':'targetbgp_prefix', 
                                   'country_y':'targetcountry' ,
                                   'registry':'targetregistry',
                                   'info_date':'targetinfo_date',
                                   'info_request':'targetinfo_request'},
                         inplace=True)
df_attacks_extended.head(5)

## 2.1.3  Lookup IP to ASN info of table: friendsenemies

In [ ]:
if (os.path.exists('enrichments/friendsenemies_iptoasn_out')== False):
    friendsenemies_iptoasn_in = open('enrichments/friendsenemies_iptoasn_in', 'w+')
    friendsenemies_iptoasn_in.write('begin\nverbose\n')
    df_friendsenemies[['ip','middledate']].drop_duplicates().to_csv(friendsenemies_iptoasn_in,header=False,index=False,sep="\t") 
    friendsenemies_iptoasn_in.write('end')
    friendsenemies_iptoasn_in.close()

    friendsenemies_iptoasn_out = open('friendsenemies_iptoasn_out', 'w+')
    iptoasn('friendsenemies_iptoasn_in',friendsenemies_iptoasn_out)
    friendsenemies_iptoasn_out.close()
else:
    print("You already performed the lookup for this table.")

In [ ]:
df_friendsenemies_iptoasn = pd.read_csv('enrichments/friendsenemies_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])

# Merging the iptoasn with the queried column
df_friendsenemies_extended= pd.merge(df_friendsenemies,
                              df_friendsenemies_iptoasn,
                              how = 'left',
                              left_on = 'ip',
                              right_on = 'ip')

# Changing name of columns to avoid misunderstandings
df_friendsenemies_extended.rename(columns={'asn':'friendsenemiesasn', 
                                   'ip':'friendsenemiesip', 
                                   'bgp_prefix':'friendsenemiesbgp_prefix', 
                                   'country':'friendsenemiescountry' ,
                                   'registry':'friendsenemiesregistry',
                                   'info_date':'friendsenemiesinfo_date',
                                   'info_request':'friendsenemiesinfo_request',
                                   'as_name': 'friendsenemiesas_name'},
                         inplace=True)

## 2.1.4  Lookup IP to ASN info of table: blacklist

In [ ]:
if (os.path.exists('enrichments/blacklist_iptoasn_out')== False):
    blacklist_iptoasn_in = open('enrichments/blacklist_iptoasn_in', 'w+')
    blacklist_iptoasn_in.write('begin\nverbose\n')
    df_blacklist[['ip','middledate']].drop_duplicates().to_csv(blacklist_iptoasn_in,header=False,index=False,sep="\t") 
    blacklist_iptoasn_in.write('end')
    blacklist_iptoasn_in.close()

    blacklist_iptoasn_out = open('blacklist_iptoasn_out', 'w+')
    iptoasn('blacklist_iptoasn_in',blacklist_iptoasn_out)
    blacklist_iptoasn_out.close()
else:
    print("You already performed the lookup for this table.")

In [ ]:
df_blacklist_iptoasn = pd.read_csv('enrichments/blacklist_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])

# Merging the iptoasn with the queried column
df_blacklist_extended= pd.merge(df_blacklist,
                              df_blacklist_iptoasn,
                              how = 'left',
                              left_on = 'ip',
                              right_on = 'ip')

# Changing name of columns to avoid misunderstandings
df_blacklist_extended.rename(columns={'asn':'blacklistasn', 
                                   'ip':'blacklistip', 
                                   'bgp_prefix':'blacklistbgp_prefix', 
                                   'country':'blacklistcountry' ,
                                   'registry':'blacklistregistry',
                                   'info_date':'blacklistinfo_date',
                                   'info_request':'blacklistinfo_request',
                                   'as_name': 'blacklistas_name'},
                         inplace=True)
df_blacklist_extended.head(5)

## 2.2.1. Check if IP was a TOR node for table: login

In [ ]:
date_tor_check

In [ ]:
if len(df_logins['userip'].unique())<1200:
    if (os.path.exists('enrichments/logins_torcheck')== False):
        print("Note: it can take a while to finish...")
        logins_torcheck = open('logins_torcheck', 'w+')
        for i in df_logins['userip'].unique():
            wasTor=WasTorNode(i,date_tor_check)
            print(i, wasTor, file=logins_torcheck)
    #         print(i, wasTor) #DEBUGING =D
            time.sleep(random.randint(1,3)) #adding some random sleep time
            logins_torcheck.flush()

        logins_torcheck.close()
    else:
        print("The IP addresses from this table were already checked.")
else:
        print("Aborted!!! It will take more than one hour to analyse!")

In [ ]:
df_logins_torcheck = pd.read_csv('enrichments/logins_torcheck',\
                          delimiter="\s+",\
                          names = ['userip', 'tor'])

## 2.2.2. Check if IP was a TOR node for table: attacks

In [ ]:
if len(df_attacks['targetip'].unique())<1200:
    if (os.path.exists('enrichments/attacks_torcheck')== False):
        print("Note: it can take a while to finish...",len(df_attacks['targetip'].unique())*3,"seconds (in the worst case).")

        attacks_torcheck = open('attacks_torcheck', 'w+')

        for i in df_attacks['targetip'].unique():
            wasTor=WasTorNode(i,date_tor_check)
            print(i, wasTor, file=attacks_torcheck)
            print(i, wasTor) #DEBUGING =D
            time.sleep(random.randint(1,3)) #adding some random sleep time
            attacks_torcheck.flush()

        attacks_torcheck.close()
    else:
        print("The IP addresses from this table were already checked.") 
else:
        print("Aborted!!! It will take more than one hour to analyse!")

In [ ]:
df_attacks_torcheck = pd.read_csv('enrichments/attacks_torcheck',\
                          delimiter="\s+",\
                          names = ['targetip', 'tor'])

## 2.2.3. Check if IP was a TOR node for table: friendsenemies

In [ ]:
if len(df_friendsenemies['ip'].unique()) <1200:
    if (os.path.exists('enrichments/friendsenemies_torcheck')== False):
        print("Note: it can take a while to finish...",len(df_friendsenemies['ip'].unique())*3,"seconds (in the worst case).")

        friendsenemies_torcheck = open('enrichments/friendsenemies_torcheck', 'w+')

        for i in df_friendsenemies['ip'].unique():
            wasTor=WasTorNode(i,date_tor_check)
            print(i, wasTor, file=friendsenemies_torcheck)
        #     print(i, wasTor) #DEBUGING =D
            time.sleep(random.randint(1,3)) #adding some random sleep time
            friendsenemies_torcheck.flush()

        friendsenemies_torcheck.close()
    else:
        print("The IP addresses from this table were already checked.") 
else:
        print("Aborted!!! It will take more than one hour to analyse!")

In [ ]:
df_friendsenemies_torcheck = pd.read_csv('enrichments/friendsenemies_torcheck',\
                          delimiter="\s+",\
                          names = ['ip', 'tor'])

## 2.2.4. Check if IP was a TOR node for table: blacklist

In [ ]:
if len(df_blacklist['ip'].unique()) < 1200:
    if (os.path.exists('enrichments/blacklist_torcheck')== False):
        print("Note: it can take a while to finish...",len(df_blacklist['ip'].unique())*3,"seconds (in the worst case).")

        blacklist_torcheck = open('enrichments/blacklist_torcheck', 'w+')

        for i in df_blacklist['ip'].unique():
            wasTor=WasTorNode(i,date_tor_check)
            print(i, wasTor, file=blacklist_torcheck)
    #         print(i, wasTor) #DEBUGING =D
            time.sleep(random.randint(1,3)) #adding some random sleep time
            blacklist_torcheck.flush()

        blacklist_torcheck.close()
    else:
        print("The IP addresses from this table were already checked.") 
else:
        print("Aborted!!! It will take more than one hour to analyse!")

In [ ]:
df_blacklist_torcheck = pd.read_csv('enrichments/blacklist_torcheck',\
                          delimiter="\s+",\
                          names = ['ip', 'tor'])

## * Relation between Attack dates and Login dates

In [ ]:
def nearestDate(base_date, date_list):
    nearest={}
    for date in date_list:
        if (base_date.timestamp() - date.timestamp())>=0:
            nearest[base_date.timestamp() - date.timestamp()]= date
    return nearest[min(nearest.keys())]

In [ ]:
#This is the TOTAL number records to be checks!!!!
len(df_attacks)

In [ ]:
df_attacks['nearestlogin']=""
df_attacks['nearestlogin']=pd.to_datetime(df_attacks['nearestlogin'])
    
if len(df_attacks)>0 and len(df_logins)>0:
    #When was the last login of the user that performed attacks
    df_attacks['nearestlogin']=""
    df_attacks['nearestlogin']=pd.to_datetime(df_attacks['nearestlogin'])

    for index, row in df_attacks.head(100).iterrows():
        intermediate_df= df_logins[df_logins['username']==row['username']]
        nearestlogindate= nearestDate(row['date'],intermediate_df['date'])
        df_attacks.set_value(index, 'nearestlogin', nearestlogindate)

        #DEBUGGING
        if index % 1000 == 0:
            print(index,": +1000 records analysed")

In [ ]:
if len(df_attacks['nearestlogin'].value_counts()) >1:
    df_attacks_and_logins = pd.merge(df_attacks_extended,
                                     df_logins_extended,
                                     how = 'left',
                                     left_on = ['username','nearestlogin'],
                                     right_on = ['username','date'])
else:
    df_attacks_and_logins=""

<h2 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
STEP 3: Automatic Analysis
<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h3> 

## * Libraries that I use to plot figs

In [ ]:
import matplotlib.pyplot as plt
from matplotlib_venn import *
%matplotlib inline

import seaborn as sns
import itertools

plt.style.use('seaborn-muted')
# plt.rcParams['font.family'] = 'serif'
# plt.rcParams['font.size'] = 12

import warnings
warnings.filterwarnings('ignore')

## A.1. Number of records per table (part of the generic Booter database schema)

In [ ]:
len(df_users),len(df_logins),len(df_attacks),len(df_payments),len(df_settings),len(df_gateways), len(df_friendsenemies),len(df_blacklist),len(df_webshells),len(df_servers),len(df_plans)

## A.2. Number of users, customers, attackers, and their intersection

In [ ]:
if len(set(df_users['userid'].unique())) > 1:
    users_set=set(df_users['userid'].unique())
else:
    users_set=set(df_users['username'].unique())
    
if len(set(df_logins['userid'].unique())) > 1:
    userslogin_set=set(df_logins['userid'].unique())
else:
    userslogin_set=set(df_logins['username'].unique())

if len(set(df_payments['userid'].unique())) > 1:
    customers_set=set(df_payments['userid'].unique())
else:
    customers_set=set(df_payments['username'].unique())

if len(set(df_attacks['userid'].unique())) > 1:
    attackers_set=set(df_attacks['userid'].unique())
else:
    attackers_set=set(df_attacks['username'].unique())

intersec_customers_attacker=pd.Series(list(customers_set.intersection(attackers_set)))
intersec_users_customers=pd.Series(list(users_set.intersection(customers_set)))
intersec_users_attackers=pd.Series(list(users_set.intersection(attackers_set)))
intersec_users_customers_attackers=pd.Series(list(users_set.intersection(customers_set).intersection(attackers_set)))

In [ ]:
len(users_set),len(userslogin_set),len(customers_set),len(attackers_set)

In [ ]:
fig = plt.figure(figsize=(4,3))
fig.suptitle('Users, Customers & Attackers', fontsize=14)

ax = plt.subplot2grid((1,1), (0,0))

venn=venn3(ax=ax,subsets = {'001':len(attackers_set)-len(intersec_customers_attacker)-len(intersec_users_attackers)+len(intersec_users_customers_attackers), 
                            '010':len(customers_set)-len(intersec_users_customers)-len(intersec_customers_attacker)+len(intersec_users_customers_attackers), 
                            '011':len(intersec_customers_attacker)-len(intersec_users_customers_attackers),
                            '100':len(users_set)-len(intersec_users_customers)-len(intersec_users_attackers)+len(intersec_users_customers_attackers),
                            '101':len(intersec_users_attackers)-len(intersec_users_customers_attackers),
                            '110':len(intersec_users_customers)-len(intersec_users_customers_attackers),
                            '111':len(intersec_users_customers_attackers)},\
          set_labels = ('Users', 'Customers','Attackers'),\
          alpha=1)
try:
    venn.get_patch_by_id('100').set_color('#404096')
except:
    print("*Users set is empty!")  
    
try:
    venn.get_patch_by_id('110').set_color('#DEA73A')
except:
    print("*Customers set is empty!")   

try:
    venn.get_patch_by_id('001').set_color('#D92120')
except:
    print("*Attackers set is empty!")

fig.show()
# fig.savefig('../figs/timeseries_attacks.eps', format='eps', dpi=1200) 

## A.3. Distribution of login times per user

In [ ]:
if len(df_logins['userid'].value_counts()) > 0:
    num_distinct_logins_per_user=df_logins['userid'].value_counts()
else:
    num_distinct_logins_per_user=df_logins['username'].value_counts()

freq_distinct_logins_per_user=num_distinct_logins_per_user.value_counts()
cum_dist_user_logins = np.linspace(0.,1.,len(num_distinct_logins_per_user))
cdf_user_logins = pd.Series(cum_dist_user_logins, index=num_distinct_logins_per_user.sort_values())

In [ ]:
if len(num_distinct_logins_per_user)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Distribution of Login Times by Users:', fontsize=14, y=1.05,x=0.35)
    
    #Plot CDF
    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_logins.plot(ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# logins")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    ax1.set_title("")
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    
    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_logins_per_user.plot(ax=ax2,kind='pie', 
                                       labels=None, 
                                       legend=False,
                                       startangle=270,
#                                        colors=sns.color_palette()
                                       )
    ax2.set_ylabel("")
    
    
    
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_distinct_logins_per_user.values)/(freq_distinct_logins_per_user.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_distinct_logins_per_user.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.5, 1.),
               fontsize=10)
    ####
    
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/login_times.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")


## 5.3. Number of Users that Login via TOR

In [ ]:
len(df_logins_torcheck[df_logins_torcheck['tor']==True]['userip'].value_counts())

## 5.4. Number of Distinct IP addresses by Users

In [ ]:
if len(df_logins.groupby(['userid','userip']).size().reset_index()['userid'].value_counts()) >0:
    num_distinct_ips_per_user=df_logins.groupby(['userid','userip']).size().reset_index()['userid'].value_counts()
else:
    num_distinct_ips_per_user=df_logins.groupby(['username','userip']).size().reset_index()['username'].value_counts()
    
freq_distinct_ips_per_user=num_distinct_ips_per_user.value_counts()
cum_dist_user_ips = np.linspace(0.,1.,len(num_distinct_ips_per_user))
cdf_user_ips = pd.Series(cum_dist_user_ips, index=num_distinct_ips_per_user.sort_values())

In [ ]:
if len(num_distinct_ips_per_user)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Distribution of Distinct IP address used by Users:', fontsize=14, y=1.05, x=0.4)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_ips.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# IPs")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)
    
    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_ips_per_user.plot(ax=ax2,kind='pie',
                                    labels=None,legend=False,
                                       startangle=270,
#                                        colors=sns.color_palette()
                                       )
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_distinct_ips_per_user.values)/(freq_distinct_ips_per_user.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_distinct_ips_per_user.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.5, 1.),
               fontsize=10)
    ####
    
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/num_ips_by_users.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")


## 5.5. Number of Payments by Users

In [ ]:
if len(df_payments['userid'].value_counts())>0:
    num_distinct_payments_per_user=df_payments['userid'].value_counts()
else:
    num_distinct_payments_per_user=df_payments['username'].value_counts()

freq_distinct_payments_per_user=num_distinct_payments_per_user.value_counts()
cum_dist_user_payments = np.linspace(0.,1.,len(num_distinct_payments_per_user))
cdf_user_payments = pd.Series(cum_dist_user_payments, index=num_distinct_payments_per_user.sort_values())

In [ ]:
if len(num_distinct_payments_per_user)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Distribution of Number of Payments by Users:', fontsize=14, y=1.05, x=0.4)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_payments.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# Payment")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)
    
    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_payments_per_user.plot(ax=ax2,kind='pie', 
                                         labels=None,legend=False,
                                         startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_distinct_payments_per_user.values)/(freq_distinct_payments_per_user.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_distinct_payments_per_user.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/payments_distribution.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")


## 5.6. Total Amount of Money Earned

In [ ]:
if len(df_payments['amountpaid']) >0:
    total_earned=df_payments['amountpaid'].values.sum()
    'US$ {:,.2f}'.format(float(total_earned))
else:
    print("Unfortunately, there is no data available!")

## 5.7. Amount of Money Paid by Users

In [ ]:
num_distinct_payments_money_per_user=df_payments['amountpaid'].value_counts()
freq_distinct_payments_money_per_user=num_distinct_payments_money_per_user.value_counts()
cum_dist_user_payments_money = np.linspace(0.,1.,len(num_distinct_payments_money_per_user))
cdf_user_payments_money = pd.Series(cum_dist_user_payments_money, index=num_distinct_payments_money_per_user.sort_values())

In [ ]:
if len(num_distinct_payments_money_per_user)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Distribution of Money Payments by Users:', fontsize=14, y=1.05, x=0.4)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_payments_money.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("$")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_payments_money_per_user.plot(ax=ax2,kind='pie', 
                                               labels=None,legend=False,
                                               startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_distinct_payments_money_per_user.values)/(freq_distinct_payments_money_per_user.values.sum())
    labels = ['${0:1.2f} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_distinct_payments_money_per_user.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.6, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/distribution_amount_paid.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")


## 5.8. Countries from where Users Access Booters

In [ ]:
logins_country_distribution_sorted = df_logins_iptoasn['country'].value_counts(sort=True,ascending=True)
logins_country_distribution = df_logins_iptoasn['country'].value_counts()

In [ ]:
if len(logins_country_distribution)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Users Accessing from Countries:', fontsize=14, y=1.05, x=0.28)

    ax1 = plt.subplot2grid((1,2), (0,0))
    logins_country_distribution_sorted.plot(ax=ax1,kind='barh')
    ax1.set_ylabel("# access")
    ax1.set_xlabel("Country")
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    logins_country_distribution.plot(ax=ax2,kind='pie', 
                                     labels=None,legend=False,
                                     startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*logins_country_distribution.values)/(logins_country_distribution.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(logins_country_distribution.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/user_countries.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")

## 5.9. Countries of Blacklisted IPs

In [ ]:
blacklist_country_distribution=df_blacklist_iptoasn['country'].value_counts()
blacklist_country_distribution_sorted=df_blacklist_iptoasn['country'].value_counts(sort=True,ascending=True)

In [ ]:
if len(blacklist_country_distribution)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Countries of blacklisted IPs', fontsize=14, y=1.05, x=0.28)

    ax1 = plt.subplot2grid((1,2), (0,0))
    blacklist_country_distribution_sorted.plot(ax=ax1,kind='barh')
    ax1.set_ylabel("# Access")
    ax1.set_xlabel("Country")
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    blacklist_country_distribution.plot(ax=ax2,kind='pie', 
                                        labels=None,legend=False,
                                        startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*blacklist_country_distribution.values)/(logins_country_distribution.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(blacklist_country_distribution.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/blacklist_countries.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")

## 5.10. Countries of Target IPs

In [ ]:
attacks_country_distribution=df_attacks_iptoasn['country'].value_counts()

In [ ]:
if len(attacks_country_distribution)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Countries of target IPs', fontsize=14, y=1.05, x=0.28)

    ax1 = plt.subplot2grid((1,2), (0,0))
    attacks_country_distribution.plot(ax=ax1,kind='bar')
    ax1.set_ylabel("# Access")
    ax1.set_xlabel("Country")
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    attacks_country_distribution.plot(ax=ax2,kind='pie', 
                                      labels=None,legend=False,
                                      startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*attacks_country_distribution.values)/(attacks_country_distribution.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(attacks_country_distribution.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/attack_countries_distribution.eps',bbox_inches='tight', format='eps', dpi=1200) 
else:
    print("Unfortunately, there is no data available!")

## 5.12. Attacks on Same Target

In [ ]:
num_attacks_on_sametarget=df_attacks['targetip'].value_counts()

freq_num_attacks_on_sametarget=num_attacks_on_sametarget.value_counts()
cum_num_attacks_on_sametarget = np.linspace(0.,1.,len(num_attacks_on_sametarget))
cdf_num_attacks_on_sametarget = pd.Series(cum_num_attacks_on_sametarget, index=num_attacks_on_sametarget.sort_values())

In [ ]:
if len(num_attacks_on_sametarget)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Attacks on Same Targets:', fontsize=14, y=1.05,x=0.28)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_num_attacks_on_sametarget.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# IPs")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_num_attacks_on_sametarget.plot(ax=ax2,kind='pie',
                                        labels=None,legend=False,
                                        startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_num_attacks_on_sametarget.values)/(freq_num_attacks_on_sametarget.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_num_attacks_on_sametarget.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()

    fig.show()
    fig.savefig('figs/attacks_on_same_target.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")

## 5.11. Attacks per day (timeseries) 

In [ ]:
if len(df_attacks)>0:
    attack_timeseries=df_attacks.set_index(['date']).groupby(pd.TimeGrouper(freq='D')).agg(['count'])['action']
    attack_mean_perday=attack_timeseries.mean()
    attack_median_perday=attack_timeseries.median()
else:
    attack_timeseries=""

In [ ]:
if len(attack_timeseries)>0:
    fig = plt.figure(figsize=(6,3))

    ax1 = plt.subplot2grid((1,1), (0,0), rowspan=2)
    attack_timeseries.plot(ax=ax1,
                           lw=2,
                           legend=False,
    #                        style='--'
                          )

    # X and Y Labels and Ticks
    ax1.set_xlabel("Time (bin=day)")
    ax1.set_ylabel("# Attacks")

    ax1.annotate(str(int(attack_median_perday[0]))+' (median)', 
                 (min(df_attacks['date']), attack_median_perday),
                 xytext=(350, -1), 
                 textcoords='offset points',
                 color='black', 
                 arrowprops=dict(arrowstyle='-|>',
                                 color='black'))
    fig.savefig('figs/attacks_timeseries.eps', bbox_inches='tight',format='eps', dpi=1200)

else:
    print("Unfortunately, there is no data available!")

## 5.12. Time Between Logins and Attacks

In [ ]:
if len(df_attacks_and_logins)>0:
    print("redo")
else:
    print("Unfortunately, there is no data available!")

## 5.13. Who attack whom? (users on the country level)

In [ ]:
if len(df_attacks_extended)>0 and len(df_logins_extended)>0:
    merged_attacks_logins = pd.merge(df_attacks_extended,
                                  df_logins_extended,
                                  how = 'left',
                                  left_on = 'date',
                                  right_on = 'date')[['targetcountry','srccountry']]

    who_against_whom = merged_attacks_logins.groupby(['targetcountry','srccountry'])\
                            .size()\
                            .reset_index()\
                            .pivot('srccountry','targetcountry',0)
else:
    who_against_whom =""

In [ ]:
if len(who_against_whom)>0:
    fig = plt.figure(figsize=(8,12))
    fig.suptitle('Countries of Target IPs', fontsize=14, y=.92)

    ax1 = plt.subplot2grid((1,1), (0,0))
    sns.set()
    sns.heatmap(who_against_whom,
                ax=ax1,
#                 cmap="YlGnBu",
    #             linewidths=.5,
    #             annot=True
                )

    ax1.set_ylabel("Source Country")
    ax1.set_xlabel("Attack Target Country")

    fig.show()
    fig.savefig('figs/who_attack_whom.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")

<h1 align="center">!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
FIFTH PART: Query Interface<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

In [ ]:
from ipywidgets import widgets, interact, Dropdown,HTML
from IPython.display import display,clear_output

In [ ]:
def search_by_userid_submit(sender):
    clear_output()
    print("Searching by id =",search_by_userid.value)
    
    if len(df_users[df_users['userid']== int(search_by_userid.value)])>0:
        display(df_users[df_logins['userid']== int(search_by_userid.value)])
    
    if len(df_logins_extended[df_logins_extended['userid']== int(search_by_userid.value)])>0:
        display(df_logins_extended[df_logins_extended['userid']== int(search_by_userid.value)])
    
    if len(df_attacks_extended[df_attacks_extended['userid']== int(search_by_userid.value)])>0:
        display(df_attacks_extended[df_attacks_extended['userid']== int(search_by_userid.value)])
    
    if len(df_payments[df_payments['userid']== int(search_by_userid.value)])>0:
        display(df_payments[df_payments['userid']== int(search_by_userid.value)])

def search_by_username_submit(sender):
    clear_output()
    print("Searching by username =",search_by_username.value,"\n")
    
    if len(df_users[df_users['username']== search_by_username.value])>0:
        print("Table df_users:")
        display(df_users[df_users['username']== search_by_username.value])
    
    if len(df_logins_extended[df_logins_extended['username']== search_by_username.value])>0:
        print("Table df_logins:")
        display(df_logins_extended[df_logins_extended['username']== search_by_username.value])

    if len(df_attacks_extended[df_attacks_extended['username']== search_by_username.value])>0:
        print("Table df_attacks:")
        display(df_attacks_extended[df_attacks_extended['username']== search_by_username.value])
    
    if len(df_payments[df_payments['username']== search_by_username.value])>0:
        print("Table df_payments:")
        display(df_payments[df_payments['username']== search_by_username.value])
        
def search_by_ip_submit(sender):
    clear_output()
    print("Searching by IP address =",str(search_by_ip.value),"\n")
    
    if len(df_logins_extended[df_logins_extended['userip']== str(search_by_ip.value)])>0:
        print("Table df_logins (as ATTACKER:")
        display(df_logins_extended[df_logins_extended['userip']== str(search_by_ip.value)])
     
    if len(df_attacks_extended[df_attacks_extended['targetip']== str(search_by_ip.value)])>0:
        print("Table df_attacks (as TARGET):")
        display(df_attacks_extended[df_attacks_extended['targetip']== str(search_by_ip.value)])  
      
       
def search_by_asn_submit(sender):
    clear_output()
    print("Searching by Autonomous System Number (ASN) =",search_by_asn.value,"\n")
    
    if len(df_logins_extended[df_logins_extended['srcasn']== int(search_by_asn.value)])>0:
        print("Table df_logins (as ATTACKER):")
        display(df_logins_extended[df_logins_extended['srcasn']== int(search_by_asn.value)])
    
    if len(df_attacks_extended[df_attacks_extended['targetasn']== str(search_by_asn.value)])>0:
        print("Table df_attacks (as TARGET):")
        display(df_attacks_extended[df_attacks_extended['targetasn']== str(search_by_asn.value)])

    if len(df_attacks_extended[df_attacks_extended['targetasn']== str(search_by_asn.value)])>0:
        print("Table df_attacks (as TARGET):")
        display(df_attacks_extended[df_attacks_extended['targetasn']== str(search_by_asn.value)])

        
country_list=pd.read_csv('https://raw.githubusercontent.com/datasets/country-list/master/data.csv',delimiter=",",error_bad_lines=False)
def search_by_country_submit():
    country_code=country_list[country_list['Name']==search_by_country.value]['Code'].values[0]
    clear_output()
    print("Searching by Country =",search_by_country.value,"\n")
    
    if len(df_logins_extended[df_logins_extended['srccountry']== country_code])>0:
        print("Table df_logins (as ATTACKER):")
        display(df_logins_extended[df_logins_extended['srccountry']== country_code])
    
    if len(df_attacks_extended[df_attacks_extended['targetcountry']== country_code])>0:
        print("Table df_attacks (as TARGET):")
        display(df_attacks_extended[df_attacks_extended['targetcountry']== country_code])
    

In [ ]:
display(HTML('<h3>*Search by (only) one field per time:</h3>'))

search_by_username = widgets.Text(description="username")
display(search_by_username)
search_by_username.on_submit(search_by_username_submit)

search_by_userid = widgets.Text(description="userid")
display(search_by_userid)
search_by_userid.on_submit(search_by_userid_submit)

search_by_ip = widgets.Text(description="IP")
display(search_by_ip)
search_by_ip.on_submit(search_by_ip_submit)

search_by_asn = widgets.Text(description="ASN")
display(search_by_asn)
search_by_asn.on_submit(search_by_asn_submit)

search_by_country = Dropdown(description="Country", options=country_list['Name'].tolist())
search_by_country.on_trait_change(search_by_country_submit, name="value")
display(search_by_country)